In [1]:
import os
import pandas as pd
import json

In [2]:
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader 
from sklearn.model_selection import train_test_split

c:\Users\kimso\anaconda3\envs\test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 데이터 불러오기

In [3]:
novel_final = pd.read_csv('./dataset/novel_final.csv')

In [63]:
data = novel_final.copy()
data = data[:int(len(data)*0.5)]

### Preprocessing

In [5]:
data.head()

,Unnamed: 0,title,text,top_sentiment,second_sentiment
0,0,이야기꾼 구연설화,"['01범보다 무서운 곶감', '화자를 처음 만나 이야기를 들으러 왔다고 하자 서슴...",분노,당황
1,1,이야기꾼 구연설화,"['<봄꿩은 제 울음에 저 죽는다>', '그 말과 같아서 사램이 잘못 되머넌 하는 ...",상처,분노
2,2,이야기꾼 구연설화,"['그래 그 여자가 쪄서 쌀얼 쪄서 밥얼 했어.', '“잡수라.”구.', '그래 한...",상처,당황
3,3,이야기꾼 구연설화,"['그랴. 그래 우리 인제 사춘 찾어간다는 얘기를 족~ 갈쳐중개,', '“그러시냐구...",상처,분노
4,4,이야기꾼 구연설화,"['나와서 인제 그 집 먼지 인저 그, 그, 뭐여 면사무소 있는 디 나와서 인제, ...",상처,분노


In [92]:
from sklearn.preprocessing import LabelEncoder

# 감정 카테고리
emotions = ['상처', '불안', '기쁨', '슬픔', '분노', '당황']

# LabelEncoder 초기화
le = LabelEncoder()
le.fit(emotions)

# 감정1과 감정2를 숫자로 변환
data['감정1_encoded'] = le.transform(data['top_sentiment'])
data['감정2_encoded'] = le.transform(data['second_sentiment'])

# 수치형 감정1, 감정2와 텍스트 결합
data['input_text'] = data['감정1_encoded'].astype(str) + " | " + data['감정2_encoded'].astype(str) + " | " + data['text']

# 불필요한 문자 제거
data['input_text'] = data['input_text'].str.replace(r"[\[\],']", "", regex=True)

data[['top_sentiment', 'second_sentiment', 'input_text']].sample(n=5)

,top_sentiment,second_sentiment,input_text
157,불안,당황,3 | 1 | “낫은 굽은 면을 다스리기 좋게 휘어 있지만 지금 너희는 그 장점을 ...
3973,불안,상처,"3 | 4 | ""물론 드래곤은 그녀의 양아버지야. 그리고 그녀를 대단히 사랑하고 있..."
1018,불안,기쁨,3 | 0 | 케이를 공항에 데려다 주었다는 말을 들으면 또 무슨 난리를 칠지 모른...
1117,불안,상처,"3 | 4 | 방문객은 담담했다. 대신 품속에서 문서를 하나 꺼냈다. ""황제의 조서..."
731,불안,당황,"3 | 1 | ""이 여자의 남자가 건방지게스리 내 자리를 노리고 있다는구만!"" ""그..."


In [65]:
# 감정별 숫자 매핑 출력
for emotion, label in zip(le.classes_, le.transform(le.classes_)):
    print(f"{emotion}: {label}")

기쁨: 0
당황: 1
분노: 2
불안: 3
상처: 4
슬픔: 5


In [66]:
# 데이터셋 클래스 정의
class NovelsDataset(Dataset):
    def __init__(self, data, tokenizer, text_column="input_text", max_length=128):
        self.data = data.reset_index(drop=True)  # 인덱스 리셋 (중복 방지)
        self.tokenizer = tokenizer
        self.text_column = text_column
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.loc[idx, self.text_column] # input_text 가져오기

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)

        # labels 생성 (input_ids 복사 및 패딩 토큰을 -100으로 설정)
        labels = input_ids.clone()
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels,
        }

### Modeling

In [67]:
# KOGPT2 모델과 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

# 모델 초기화
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.bos_token_id = tokenizer.bos_token_id

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [68]:
dataset = NovelsDataset(data, tokenizer)
dataloader = DataLoader(dataset, batch_size=2)

In [69]:
# 학습 인자 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    save_steps=50,
    save_total_limit=2,
    logging_steps=10,
    eval_strategy="epoch",
)

In [70]:
train_novels, eval_novels = train_test_split(data, test_size=0.1, random_state=42)

train_dataset = NovelsDataset(train_novels, tokenizer)
eval_dataset = NovelsDataset(eval_novels, tokenizer)

In [71]:
# Trainer에 데이터셋 전달
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # 검증 데이터셋
)

In [72]:
sample = train_dataset[0]
print("Input IDs:", sample['input_ids'])
print("Attention Mask:", sample['attention_mask'])
print("Labels:", sample['labels'])

Input IDs: tensor([ 9085,   739,   466, 10595,   739,   466, 10063,  8046,   406, 10401,
         9198,  7895, 23775,   406,   377, 10063, 21319,  6889, 17272,  9269,
        12679, 40422, 14636, 15255,  7769,  7048,  7285, 11218,   406,   377,
         9054, 14669, 15728, 14243,  9860, 31427,  9342,  7610,  8711, 34692,
        10769, 15255,  7769,  7048,  7162, 24444, 10351, 11993,  9137,  8563,
         8705, 11688,  9712, 33930,  9661,  9399,  9769,  8359,  8704, 10652,
        47980,  8811,  9071, 24444, 10185, 10644,   739,  7705,  8811, 12916,
        22870,  7530,  9054, 18180,  9215,  7545, 27244,  9135, 13872, 10063,
         8015,  9217,  7162, 11242,  9109,   389,  9093,  8529, 11727,  9040,
        21713, 17044,  6897,  8146, 17885, 10704, 12205,  8711,   389,  9042,
        25831, 17582, 36334, 33016,  9122,  9329,  9658,  8718,  7055,   406,
          377, 10063,  8185, 29045,  9022,  6866, 13568, 47711, 10063, 43056,
         9022,  7123,  7285, 14096,  9063,  9038, 114

In [73]:
# 모델 학습
trainer.train()

Epoch,Training Loss,Validation Loss
0,4.206500,4.162277
1,3.736700,4.153172
2,3.402300,4.189591
3,2.968600,4.243047
4,2.769700,4.280783


TrainOutput(global_step=2520, training_loss=3.4456768800341893, metrics={'train_runtime': 658.6562, 'train_samples_per_second': 30.646, 'train_steps_per_second': 3.826, 'total_flos': 1317434425344000.0, 'train_loss': 3.4456768800341893, 'epoch': 4.99851411589896})

In [74]:
device = torch.device("cuda")

# 이야기 생성 함수
def generate_story(lyrics_input, tag1="2", tag2="4"):
    # 감정 태그를 앞에 추가하여 모델에 전달
    emotion_input = f"{tag1} | {tag2} | {lyrics_input}"

    # 입력 토큰화
    input_ids = tokenizer.encode(emotion_input, return_tensors='pt').to(device)

    # 출력 생성
    output = model.generate(
        input_ids, 
        max_new_tokens=150, # 새로 생성할 토큰의 개수를 제한
        num_return_sequences=1, 
        temperature=0.8, 
        top_k=50, 
        top_p=0.9, 
        repetition_penalty=1.2, 
        do_sample=True  # 샘플링 활성화
    )

    # 입력 길이 추적
    input_length = input_ids.shape[1]

    # 생성된 토큰 중 입력 토큰 이후의 부분만 디코딩
    generated_tokens = output[0][input_length:]
    generated_story = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    return generated_story

In [83]:
# 가사로 이야기 생성
lyrics_input = "어쩌다 고작 그 마음도 못 참고 멍청하게 다 던졌는지 뭔가 들켜 버린 것 같아 표정을 보니 말이야 나도 티가 나버린 고백에 얼마나 놀랐는지 몰라 매일 치는 장난에도 두근댔고 오늘도 몇 번이고 떨렸지만 약속했어 날 안아줘 좀 알아줘 이건 꿈에서만 하기야 무심코 던진 니 말에 하루 종일 설레어 간직했다 아무도 못 보게 일기장에 적어 단단히 잠궜었는데 어쩌다 고작 그 마음도 못 참고 멍청하게 다 던졌는지 꾹꾹 참고 또 꼭꼭 숨겨서 이제까지 잘해 왔잖아 그러다 고작 울음도 못 참고 괜찮다 말하며 두 눈은 퉁퉁 붓고 코맹맹이가 되어도 난 내일은 맑음 예전처럼 옆에서 밥 먹어도 우연히 눈이 살짝 마주쳐도 걱정 마 날 안아줘 아니 사랑해줘 이건 꿈에서만 하니까 무심코 던진 니 말에 하루 종일 설레어 간직했다 아무도 못 보게 꼬깃꼬깃 구겨 씹어 다 삼켰었는데 어쩌다 고작 그 마음도 못 참고 멍청하게 다 던졌는지 꾹꾹 참고 또 꼭꼭 숨겨서 이제까지 잘 해 왔잖아 그러다 고작 울음도 못 참고 괜찮다 말하며 두 눈은 퉁퉁 붓고 코맹맹이가 되어도 난 사실 나 아주 오래 울 것 같아 고작 친구도 못 되니까 툭툭 털고 활짝 웃을 만큼 나는 그리 강하지가 않아 그러다 고작 사랑이 뭐라고 괜찮다 말하는 날까지 꾹꾹 참고 또 일기나 쓰고 있어 나 내 이름 맑음"
generated_story = generate_story(lyrics_input, tag1="2", tag2="2")
print("생성된 이야기:", generated_story)

생성된 이야기: ~ 라는 말을 꺼내지도 못하고 이러지 말끔히 털어놓고 눈 감고 잠잠깐 감춰둔 채 눈물만 하면 이제 그만 잠그려 그런 나를 속삭여 알았으니 그때를 그리워하는 건데 내가 평생 잊게 만든 내 인생이라는 걸 후회하며 톡 까마져 버릴 줄줄라 잔소리 한번 제대로 하지 못했었으면 좋겠네 하고 말이지 이렇게 말해 놓고 가만히 생각해 봐 이런 내가 한 번도 못해 너 때문에 참을 수 없었던 게 어쩌면 네 생각뿐이라며 왜 그랬더니... 라고 할 수만 있다면 진짜로 넌 정말 몰랐다면 정말 바보라는 말 한마디라도 해야 하는 거라며.... 이젠 더 이상 나의 마음을 어떻게 대처할까 고민하고 살았다고 해도 나 지금 사라질
